In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.base import TransformerMixin, BaseEstimator

from sklearn.model_selection import StratifiedKFold

from xgboost import XGBClassifier
from sklearn.metrics import ndcg_score



In [2]:
session=pd.read_csv("C:/Users/USER/Downloads/sessions.csv/sessions.csv")
print(session.shape)
session.head()

(10567737, 6)


,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [3]:
train_user=pd.read_csv("C:/Users/USER/Downloads/train_users_2.csv/train_users_2.csv")
print(train_user.shape)
train_user.head()

(213451, 16)


,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [4]:
test_user=pd.read_csv("C:/Users/USER/Downloads/test_users.csv/test_users.csv")
print(test_user.shape)
test_user.head()

(62096, 15)


,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,20140701000051,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,20140701000148,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
3,6c6puo6ix0,2014-07-01,20140701000215,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
4,czqhjk3yfe,2014-07-01,20140701000305,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari


In [5]:
class Custom_Proccess(BaseEstimator, TransformerMixin):
   
    def transform(self,X,y=None):

        #replacing the null values 
        
        X.gender.replace('-unknown-', 'OTHER', inplace=True)
        X['age'].fillna(-1,inplace=True)
        
        X['timestamp_first_active']=X['timestamp_first_active'].apply(lambda s:datetime(year=int(str(s)[0:4]), month=int(str(s)[4:6]), 
                                                                                          day=int(str(s)[6:8])).strftime('%Y-%m-%d'))
        
        X['timestamp_first_active']=X['timestamp_first_active'].astype('datetime64[ns]')
        X['age']=X['age'].astype('int64')
        X['date_account_created']=X['date_account_created'].astype('datetime64[ns]')

        X['dac_year']=X['date_account_created'].dt.year
        X['dac_month']=X['date_account_created'].dt.month
        X['dac_day']=X['date_account_created'].dt.day

        X['tfa_year']=X['timestamp_first_active'].dt.year
        X['tfa_month']=X['timestamp_first_active'].dt.month
        X['tfa_day']=X['timestamp_first_active'].dt.day
          
            
            
            ## Removing wrong age
        ###X=X[(X.age>10)&(X.age<100)|(X.age==-1)].reset_index(drop=True)
        

        X.signup_app.replace(['iOS','Android'],'SmartDevice',inplace=True)

        X.drop(['date_first_booking','date_account_created','timestamp_first_active','first_device_type','first_browser'],axis=1,inplace=True)
        
        return X
    
    def fit(self, X, y=None, **fit_params):
        return self



In [6]:
session.drop(['action_detail','device_type'],inplace=True,axis=1)
session.dropna(subset=['user_id','action'],inplace=True)
session.action_type=session.action_type.fillna('Other')
session.secs_elapsed=session.secs_elapsed.fillna(0)

Here i have dropped the unwanted values and droped the Null values in the user_id column and action column

In [7]:
session_group=session.groupby(['user_id','action_type']).agg({'action':'count','secs_elapsed':'sum'}).reset_index()
session_df=pd.get_dummies(session_group,columns=['action_type']).groupby(['user_id']).sum().reset_index()
session_df.head()

,user_id,action,secs_elapsed,action_type_-unknown-,action_type_Other,action_type_booking_request,action_type_booking_response,action_type_click,action_type_data,action_type_message_post,action_type_modify,action_type_partner_callback,action_type_submit,action_type_view
0,00023iyk9l,39,738079.0,0,1,1,0,1,1,0,0,1,0,1
1,0010k6l0om,63,586543.0,1,1,0,0,1,1,0,0,1,0,1
2,001wyh0pz8,90,282965.0,1,1,0,0,1,1,0,0,0,1,1
3,0028jgx1x1,31,297010.0,1,0,0,0,1,1,0,0,0,1,1
4,002qnbzfs5,782,6463327.0,1,1,1,0,1,1,1,0,0,1,1


In [8]:
train_user_df=train_user.merge(session_df,left_on=['id'],
                               right_on=['user_id'],how='left').drop(['user_id'],axis=1).reset_index(drop=True)
train_user_df.shape

(213451, 29)

In [9]:
test_user_df=test_user.merge(session_df,left_on=['id'],
                               right_on=['user_id'],how='left').drop(['user_id'],axis=1).reset_index(drop=True)
test_user_df.shape

(62096, 28)

In [10]:
train_user_df.secs_elapsed.fillna(-1,inplace=True)
train_user_df.action.fillna(-1,inplace=True)
train_user_df.iloc[:,-11:]=train_user_df.iloc[:,-11:].fillna(-1)

train_user_df['secs_elapsed']=train_user_df['secs_elapsed'].astype('int64')
train_user_df['action']=train_user_df['action'].astype('int64')

In [11]:
test_user_df.secs_elapsed.fillna(-1,inplace=True)
test_user_df.action.fillna(-1,inplace=True)
test_user_df.iloc[:,-11:]=test_user_df.iloc[:,-11:].fillna(-1)

test_user_df['secs_elapsed']=test_user_df['secs_elapsed'].astype('int64')
test_user_df['action']=test_user_df['action'].astype('int64')

## Finding the shape of the train and test data

In [12]:
train_user_df.isnull().sum()/train_user_df.shape[0] *100

id                               0.000000
date_account_created             0.000000
timestamp_first_active           0.000000
date_first_booking              58.347349
gender                           0.000000
age                             41.222576
signup_method                    0.000000
signup_flow                      0.000000
language                         0.000000
affiliate_channel                0.000000
affiliate_provider               0.000000
first_affiliate_tracked          2.841402
signup_app                       0.000000
first_device_type                0.000000
first_browser                    0.000000
country_destination              0.000000
action                           0.000000
secs_elapsed                     0.000000
action_type_-unknown-            0.000000
action_type_Other                0.000000
action_type_booking_request      0.000000
action_type_booking_response     0.000000
action_type_click                0.000000
action_type_data                 0

Here i have multiplied 100 because the total number of missing values is very large the numbers will be shown large so if i multiply with 100 the values will be printed in decimal values 

## Seperating the categorical and the numerical values

In [13]:
categorical_cols=[cname for cname in train_user_df.columns if cname not in ['id','date_account_created','date_first_booking','first_device_type','first_browser',
                                                                            'timestamp_first_active','country_destination'] and 
                  train_user_df[cname].dtype == "object"]

numerical_cols=[cname for cname in train_user_df.columns if cname not in ['id','date_account_created','date_first_booking','first_device_type','first_browser',
                                                                          'timestamp_first_active','country_destination'] and 
                train_user_df[cname].dtype != "object"]

print("Categorical_cols - \n",categorical_cols)
print("Numerical_cols - \n",numerical_cols)

Categorical_cols - 
 ['gender', 'signup_method', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app']
Numerical_cols - 
 ['age', 'signup_flow', 'action', 'secs_elapsed', 'action_type_-unknown-', 'action_type_Other', 'action_type_booking_request', 'action_type_booking_response', 'action_type_click', 'action_type_data', 'action_type_message_post', 'action_type_modify', 'action_type_partner_callback', 'action_type_submit', 'action_type_view']


In [14]:
# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocessing for categorical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols),
        ('num', numerical_transformer, numerical_cols)
    ])

In [15]:
test_id = test_user_df.id
test_X = test_user_df.drop(['id'], axis='columns')

In [16]:
labels = train_user_df.country_destination
le = LabelEncoder()
train_y = le.fit_transform(labels) 

## Seperating tbhe dataset into train and test and finding their shape to check if they match 

In [17]:
train_X = train_user_df.drop(['id','country_destination'], axis='columns')

In [18]:
train_X.shape,train_user_df.shape,len(train_y),train_user.shape

((213451, 27), (213451, 29), 213451, (213451, 16))

In [19]:
test_user_df.shape,test_X.shape

((62096, 28), (62096, 27))

In [20]:
def cross_validation_with_ndcg(pipe, X, y, scorer, cv=5):
    
    #we use StratifiedKFold to ensure that the train and test data is being seperated in the same proportion 
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=100)
    
    # initialize the values 
    scores = []
    for train_index, holdout_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[holdout_index]
        y_train, y_test = y[train_index], y[holdout_index]
        pipe.fit(X_train, y_train)
        predict = pipe.predict_proba(X_test)
        y_test = pd.get_dummies(y_test).to_numpy()
        score = scorer(y_test, predict)
        scores.append(round(score, 6))
        print(f'{len(scores)} / {cv} DONE!', end='\r')
    return scores

## Implementatio of XgBoost Classifier model 

First we are setting the parameters for the model to train

Then we are applying the model here to find the accuracy 


In [21]:
#grid search

#here we are assigining the parameters for the model training
n_estimaters_param=[50, 100, 200]
max_depth_param=[3,4,5]
learning_rate_param=[0.1,0.2]

params = [(x, y, z) for x in learning_rate_param for y in n_estimaters_param for z in max_depth_param]

result_list=[]


for learning_rates,n_estimaters, max_depth in params:
    xg_model_ = XGBClassifier(max_depth=max_depth,learning_rate=learning_rates, n_estimators=n_estimaters,verbosity=0,
                              objective='multi:softprob',n_jobs=-1)         #Xgboost model
    search_pipe = Pipeline([
        ('customproccess',Custom_Proccess()),
        ('preprocessor', preprocessor),
        ("model", xg_model_)
    ])                   #assembling the steps that can be crossvalidated 
    
    print(f'learning_rate: {learning_rates}, n_estimaters: {n_estimaters}, max_depth: {max_depth}')
    scores = cross_validation_with_ndcg(search_pipe, train_X, train_y, ndcg_score)
    
    #we are 
    result_list.append([learning_rates,n_estimaters,max_depth,np.mean(scores)])

learning_rate: 0.1, n_estimaters: 50, max_depth: 3


C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.gender.replace('-unknown-', 'OTHER', inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['age'].fillna(-1,inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.gender.replace('-unknown-', 'OTHER', inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['age'].fillna(-1,inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.gender.replace('-unknown-', 'OTHER', inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['age'].fillna(-1,inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.gender.replace('-unknown-', 'OTHER', inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['age'].fillna(-1,inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

...

C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.gender.replace('-unknown-', 'OTHER', inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['age'].fillna(-1,inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.gender.replace('-unknown-', 'OTHER', inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['age'].fillna(-1,inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.gender.replace('-unknown-', 'OTHER', inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['age'].fillna(-1,inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.gender.replace('-unknown-', 'OTHER', inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['age'].fillna(-1,inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.gender.replace('-unknown-', 'OTHER', inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['age'].fillna(-1,inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_4520\793395875.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [22]:
result_df=pd.DataFrame(result_list,columns=['learning_rate','n_estimator','max_depth','mean_score'])
result_df.sort_values(by='mean_score',ascending=False).head(5)

,learning_rate,n_estimator,max_depth,mean_score
8,0.1,200,5,0.843493
11,0.2,50,5,0.843487
5,0.1,100,5,0.843478
16,0.2,200,4,0.843464
13,0.2,100,4,0.843381


Here we have found the accuracy and the highest we have got is 84%.

If we change the parameters and increase the number of epoch we mayincrease the accuracy of the model.